# Experiment 4: RAG + Tool Use

In [1]:
# Setup
import sys
import json
import os
from pathlib import Path
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Wolfram Alpha
import requests

# RAG
import chromadb
from chromadb.utils import embedding_functions

# LLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

sys.path.append('..')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Imports loaded")

Imports loaded


In [2]:
# Configuration
DB_PATH = Path("../data/vector_db")
MODEL_PATH = Path("/home/sskaplun/study/genAI/kaggle/models/gemma-2-9b-it")
OUTPUT_DIR = Path("../evaluation/experiment_04")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

COLLECTION_NAME = "ukrainian_math"
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

# Wolfram Alpha API
WOLFRAM_APP_ID = 'YL2L8P8W5J'

# RAG parameters
TOP_K = 5
TEMPERATURE = 0.7
MAX_NEW_TOKENS = 600

print(f"Wolfram API: {'Configured' if WOLFRAM_APP_ID != 'DEMO' else 'DEMO mode (limited)'}")
print(f"CUDA: {torch.cuda.is_available()}")

Wolfram API: Configured
CUDA: True


In [3]:
@dataclass
class ToolCall:
    tool_name: str
    query: str
    result: str
    success: bool

@dataclass
class RAGToolResponse:
    question: str
    answer: str
    citations: List[str]
    tool_calls: List[ToolCall]  # NEW: track tool usage
    avg_relevance: float
    answer_length: int
    verified: bool  # NEW: was answer computationally verified?
    
    def to_dict(self):
        return {
            'question': self.question,
            'answer': self.answer,
            'citations': self.citations,
            'tool_calls': [asdict(t) for t in self.tool_calls],
            'avg_relevance': self.avg_relevance,
            'answer_length': self.answer_length,
            'verified': self.verified
        }

print("Dataclasses defined")

Dataclasses defined


## 1. Load Vector Database

In [4]:
print("="*80)
print("LOADING VECTOR DATABASE")
print("="*80)

client = chromadb.PersistentClient(path=str(DB_PATH))
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_MODEL
)
collection = client.get_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_function
)

print(f"Collection loaded: {collection.count():,} chunks")

LOADING VECTOR DATABASE


Collection loaded: 15,836 chunks


## 2. Load LLM

In [5]:
print("="*80)
print("LOADING LLM")
print("="*80)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(str(MODEL_PATH))
model = AutoModelForCausalLM.from_pretrained(
    str(MODEL_PATH),
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)

print("Model loaded")

LOADING LLM


The tokenizer you are loading from '/home/sskaplun/study/genAI/kaggle/models/gemma-2-9b-it' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded


## 3. Wolfram Alpha Tool

In [6]:
def query_wolfram_alpha(query: str, timeout: int = 10) -> ToolCall:
    """
    Query Wolfram Alpha API for mathematical computations.
    
    Args:
        query: Natural language or symbolic math query
        timeout: Request timeout in seconds
    
    Returns:
        ToolCall with result or error message
    """
    if WOLFRAM_APP_ID == 'DEMO':
        # DEMO mode: simulate responses
        return ToolCall(
            tool_name="wolfram_alpha",
            query=query,
            result=f"DEMO: Would compute '{query}' (set WOLFRAM_APP_ID for real queries)",
            success=False
        )
    
    try:
        # Wolfram Alpha Simple API (HTTPS required)
        url = "https://api.wolframalpha.com/v1/result"
        params = {
            'appid': WOLFRAM_APP_ID,
            'i': query
        }
        
        response = requests.get(url, params=params, timeout=timeout)
        
        if response.status_code == 200:
            result = response.text
            return ToolCall(
                tool_name="wolfram_alpha",
                query=query,
                result=result,
                success=True
            )
        else:
            return ToolCall(
                tool_name="wolfram_alpha",
                query=query,
                result=f"Error {response.status_code}: {response.text}",
                success=False
            )
    
    except Exception as e:
        return ToolCall(
            tool_name="wolfram_alpha",
            query=query,
            result=f"Exception: {str(e)}",
            success=False
        )

print("Wolfram Alpha tool defined")

Wolfram Alpha tool defined


In [7]:
# Test Wolfram Alpha
test_queries = [
    "integrate x^2",
    "solve x^2 + 5x + 6 = 0",
    "volume of sphere with radius 5"
]

print("Testing Wolfram Alpha API:")
print("="*80)
for query in test_queries:
    result = query_wolfram_alpha(query)
    print(f"\nQuery: {query}")
    print(f"Success: {result.success}")
    print(f"Result: {result.result}")

Testing Wolfram Alpha API:



Query: integrate x^2
Success: True
Result: x^3/3



Query: solve x^2 + 5x + 6 = 0
Success: True
Result: the 1st is x equals negative 3 and the 2nd is x equals negative 2



Query: volume of sphere with radius 5
Success: True
Result: (500 pi)/3


## 4. RAG + Tool Pipeline

In [8]:
def retrieve_chunks(query: str, k: int = TOP_K) -> tuple:
    """Retrieve from vector DB."""
    results = collection.query(query_texts=[query], n_results=k)
    
    chunks = []
    citations = []
    
    for doc, meta, dist in zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    ):
        citation = f"[{meta['filename']}, с. {meta['page_start']}-{meta['page_end']}]"
        header = f"[Джерело {len(chunks)+1}] {citation} | Тип: {meta['content_type']}"
        chunks.append(f"{header}\n{doc}")
        citations.append(citation)
    
    context = "\n\n".join(chunks)
    avg_relevance = float(np.mean([1 - d for d in results['distances'][0]]))
    
    return context, citations, avg_relevance

print("Retrieval function defined")

Retrieval function defined


In [9]:
SYSTEM_PROMPT = """Ти — досвідчений викладач математики для українських учнів 10-11 класів.

Ти маєш доступ до інструментів:
1. **Підручники** (контекст нижче) - для теорії та формул
2. **Wolfram Alpha** - для обчислень та перевірки

Твоє завдання:
- Згенерувати математичну задачу з ПЕРЕВІРЕНИМ розв'язанням
- Використовувати формули з підручників
- ОБОВ'ЯЗКОВО використати Wolfram Alpha для перевірки обчислень
- Надати крок-за-кроком розв'язання українською мовою

Як використовувати Wolfram Alpha:
Напиши: [WOLFRAM: твій запит]
Приклад: [WOLFRAM: volume of sphere with radius 5]

Формат відповіді:
**Задача:** [текст задачі]

**Розв'язання:**
1. [крок 1]
2. [крок 2]
[WOLFRAM: обчислення для перевірки]
3. [крок 3]

**Відповідь:** [фінальна відповідь]

ВАЖЛИВО: 
- Використай контекст з підручників
- ОБОВ'ЯЗКОВО використай Wolfram Alpha хоча б раз
- Відповідай ТІЛЬКИ українською"""

print("System prompt defined")

System prompt defined


In [10]:
def extract_wolfram_queries(text: str) -> List[str]:
    """Extract Wolfram Alpha queries from text."""
    pattern = r'\[WOLFRAM:\s*([^\]]+)\]'
    matches = re.findall(pattern, text, re.IGNORECASE)
    return [m.strip() for m in matches]

def generate_with_llm(prompt: str) -> str:
    """Generate text using LLM."""
    messages = [{"role": "user", "content": prompt}]
    formatted = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=TEMPERATURE,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    ).strip()

print("Helper functions defined")

Helper functions defined


In [11]:
def rag_tool_generate(
    question: str,
    verbose: bool = False
) -> RAGToolResponse:
    """
    RAG + Tool pipeline:
    1. Retrieve context from textbooks
    2. Generate initial answer with tool calls
    3. Execute Wolfram queries
    4. Refine answer with tool results
    """
    if verbose:
        print(f"\nQuestion: {question}")
        print("  Step 1: Retrieving context...")
    
    # 1. Retrieve
    context, citations, avg_relevance = retrieve_chunks(question)
    
    if verbose:
        print(f"    Retrieved {len(citations)} chunks")
        print("  Step 2: Generating with tool calls...")
    
    # 2. Generate with tool instructions
    prompt = f"{SYSTEM_PROMPT}\n\nКОНТЕКСТ З ПІДРУЧНИКІВ:\n{context}\n\nЗАПИТАННЯ:\n{question}\n\nТВОЯ ВІДПОВІДЬ:"
    answer = generate_with_llm(prompt)
    
    if verbose:
        print(f"    Generated {len(answer)} chars")
        print("  Step 3: Extracting tool calls...")
    
    # 3. Extract and execute Wolfram queries
    wolfram_queries = extract_wolfram_queries(answer)
    tool_calls = []
    
    if verbose and wolfram_queries:
        print(f"    Found {len(wolfram_queries)} Wolfram queries")
    
    for wq in wolfram_queries:
        if verbose:
            print(f"      Querying: {wq}")
        tool_call = query_wolfram_alpha(wq)
        tool_calls.append(tool_call)
        
        # Replace placeholder with result
        placeholder = f"[WOLFRAM: {wq}]"
        replacement = f"[WOLFRAM RESULT: {tool_call.result}]"
        answer = answer.replace(placeholder, replacement)
    
    verified = any(tc.success for tc in tool_calls)
    
    if verbose:
        print(f"    Verified: {verified}")
    
    return RAGToolResponse(
        question=question,
        answer=answer,
        citations=citations,
        tool_calls=tool_calls,
        avg_relevance=avg_relevance,
        answer_length=len(answer),
        verified=verified
    )

print("RAG+Tool pipeline defined")

RAG+Tool pipeline defined


## 5. Test Questions

In [12]:
from common import TOOL_TEST_QUESTIONS, EVALUATION_DATASET

TEST_QUESTIONS = TOOL_TEST_QUESTIONS
print(f"Test set: {len(TEST_QUESTIONS)} questions")

# Create mapping of questions to expected answers
question_to_expected = {q['input']: q['expected_answer'] for q in EVALUATION_DATASET}
print(f"Expected answers loaded for {len(question_to_expected)} questions")

Test set: 6 questions
Expected answers loaded for 10 questions


## 6. Run Experiment

In [13]:
print("="*80)
print("RUNNING RAG + TOOL USE EXPERIMENT")
print("="*80)

responses = []

for i, question in enumerate(TEST_QUESTIONS, 1):
    print(f"\n[{i}/{len(TEST_QUESTIONS)}] {question}")
    print("-"*80)
    
    response = rag_tool_generate(question, verbose=True)
    responses.append(response)
    
    print(f"\nAnswer:\n{response.answer}")
    print(f"\nTool Calls: {len(response.tool_calls)}")
    for tc in response.tool_calls:
        print(f"  - {tc.tool_name}: {tc.query[:50]}... → {tc.success}")
    print(f"\nVerified: {response.verified}")

print(f"\n{'='*80}")
print(f"Completed {len(responses)} RAG+Tool responses")
print("="*80)

RUNNING RAG + TOOL USE EXPERIMENT

[1/6] Площа трикутника
--------------------------------------------------------------------------------

Question: Площа трикутника
  Step 1: Retrieving context...
    Retrieved 5 chunks
  Step 2: Generating with tool calls...


    Generated 872 chars
  Step 3: Extracting tool calls...
    Found 1 Wolfram queries
      Querying: 4 * 3.14159 * 8^2


    Verified: True

Answer:
**Задача:**  Рівнобедреному трикутнику з основою 12 см і бічними сторонами 10 см обертаємо навколо прямої, що проходить через вершину кута при основі перпендикулярно до основи. Знайдіть площу поверхні отриманої сфери.

**Розв'язання:**
1.  Спочатку знайдемо висоту рівнобедреного трикутника. Висота є також медіаною та бісектрисою основи, тому вона ділить основу на дві частини по 6 см. 
2.  Висоту можна знайти за допомогою теореми Піфагора: 
*  a² + b² = c², де a = 6 см (половина основи), b - висота, c = 10 см (бічна сторона).
*  6² + b² = 10²
*  36 + b² = 100
*  b² = 64
*  b = 8 см
3.  Радіус сфери дорівнює висоті рівнобедреного трикутника, тобто 8 см.
4.  Площа поверхні сфери обчислюється за формулою: 4πr², де r - радіус сфери.
[WOLFRAM RESULT: 804.247]
5.  Площа поверхні сфери = 803.84 см²

**Відповідь:** Площа поверхні отриманої сфери дорівнює 803,84 см².

Tool Calls: 1
  - wolfram_alpha: 4 * 3.14159 * 8^2... → True

Verified: True

[2/6] Площа трикутника


    Generated 949 chars
  Step 3: Extracting tool calls...
    Found 1 Wolfram queries
      Querying: sqrt((10^2)-(6^2))


    Verified: True

Answer:
**Задача:** 

Рівнобедрений трикутник ABC з основою 12 см та бічними сторонами 10 см обертається навколо прямої, яка проходить через вершину кута при основі перпендикулярно до основи. Знайдіть площу поверхні та об'єм отриманої поверхні обертання.

**Розв'язання:**

1. **Знайдемо радіус сфери:** 
   - Оскільки трикутник рівнобедрений, його висота також є і бісектором кута при основі.
   - Висота трикутника ділить його на два прямокутні трикутники.
   - Використовуючи теорему Піфагора, знаходимо висоту трикутника (а отже, радіус сфери): [WOLFRAM RESULT: 8] = 8 см.

2. **Обчислимо площу поверхні сфери:**
   - Площа поверхні сфери обчислюється за формулою: S = 4πr², де r - радіус сфери.
   - S = 4 * π * (8 см)² = 256π см²

3. **Обчислимо об'єм сфери:**
   - Об'єм сфери обчислюється за формулою: V = (4/3)πr³.
   - V = (4/3) * π * (8 см)³ = 2144/3 π см³

**Відповідь:** Площа поверхні сфери дорівнює 256π см², а об'єм - 2144/3 π см³.

Tool Calls: 1
  - wolfram_alpha

...

      Querying: (4/3) * pi * 2^3


    Verified: True

Answer:
**Задача:**  У кулю вписано правильну трикутну піраміду, сторона основи якої дорівнює 6 см, а бічне ребро утворює з площиною основи кут 60°. Знайдіть об'єм кулі.

**Розв'язання:**

1. **Знайдемо висоту піраміди:**
    -  Висота піраміди  - це перпендикуляр, проведений з вершини піраміди до площини основи.
    -  За формулою  sin(α) = (h/l), де α = 60°, l - бічне ребро, h - висота піраміди.
    -  [WOLFRAM RESULT: Error 501: "No short answer available"]  => h = 6 * sin(60°) = 6 * √3 / 2 = 3√3 см

2. **Знайдемо радіус кулі:**
    -  Радіус кулі дорівнює радіусу кола, вписаного в основу трикутної піраміди.
    -  Основа піраміди - рівносторонній трикутник, тому радіус кола дорівнює 1/3 сторони основи.
    -  Радіус кулі: r = 1/3 * 6 см = 2 см

3. **Обчислимо об'єм кулі:**
    -  Об'єм кулі за формулою V = (4/3) * π * r^3
    -  [WOLFRAM RESULT: (32 pi)/3] => V = (4/3) * π * 8 = (32/3) * π см^3

**Відповідь:** Об'єм кулі дорівнює (32/3) * π см^3.

Tool Calls: 2


    Generated 984 chars
  Step 3: Extracting tool calls...
    Found 2 Wolfram queries
      Querying: sin(60)


      Querying: (4/3) * pi * ((3*sqrt(3))/2)^3


    Verified: True

Answer:
**Задача:**  У кулю вписано правильну трикутну піраміду, сторона основи якої дорівнює 6 см, а бічне ребро утворює з площиною основи кут 60°. Знайдіть об'єм кулі.

**Розв'язання:**

1. **Знайдемо висоту піраміди:**
   - У прямокутному трикутнику, утвореному бічним ребром піраміди, половиною її основи та висотою, кут при вершині дорівнює 60°, а гіпотенуза (бічне ребро) дорівнює 6 см. 
   - Використовуючи формулу  sin(α) = протилежний катет / гіпотенуза, знаходимо висоту піраміди: sin(60°) = висота / 6 см. 
   -  [WOLFRAM RESULT: sqrt(3)/2]  - sin(60°) = √3 / 2, отже висота = (√3 / 2) * 6 см = 3√3 см.

2. **Знайдемо радіус кулі:**
   - Радіус кулі дорівнює довжині половини висоти піраміди. 
   - Радіус = висота / 2 = 3√3 см / 2 = (3√3) / 2 см.

3. **Обчислимо об'єм кулі:**
   - Формула об'єму кулі: V = (4/3) * π * r³, де r - радіус.
   -  [WOLFRAM RESULT: (27 sqrt(3) pi)/2] - підставляємо радіус і обчислюємо.


**Відповідь:** Об'єм кулі дорівнює  (4/3) * π * ((

## 7. Evaluation

In [14]:
import common

print("Evaluation functions loaded from common.py")

Evaluation functions loaded from common.py


In [15]:
# Evaluate
print("="*80)
print("EVALUATION")
print("="*80)

evaluations = []

for i, response in enumerate(responses, 1):
    expected_answer = question_to_expected.get(response.question, None)
    metrics = common.evaluate_rag_tools(
        response.answer, 
        response.answer_length, 
        response.avg_relevance, 
        len(response.tool_calls) > 0, 
        response.verified,
        expected_answer
    )
    evaluations.append({
        'question': response.question,
        'metrics': metrics,
        'answer_length': response.answer_length,
        'num_tool_calls': len(response.tool_calls)
    })
    
    print(f"\n{i}. {response.question[:50]}...")
    print(f"   Overall: {metrics['overall_score']:.3f} | "
          f"Tools: {metrics['tool_usage']} | "
          f"Verified: {metrics['verified']}")

# Summary
print(f"\n{'='*80}")
print("SUMMARY")
print("="*80)

avg_metrics = {
    'overall_score': np.mean([e['metrics']['overall_score'] for e in evaluations]),
    'retrieval_quality': np.mean([e['metrics']['retrieval_quality'] for e in evaluations]),
    'ukrainian_ratio': np.mean([e['metrics']['ukrainian_ratio'] for e in evaluations]),
    'completeness': np.mean([e['metrics']['completeness'] for e in evaluations]),
    'correctness': np.mean([e['metrics']['correctness'] for e in evaluations]),
    'structure_rate': sum(e['metrics']['has_structure'] for e in evaluations) / len(evaluations),
    'citation_rate': sum(e['metrics']['has_citations'] for e in evaluations) / len(evaluations),
    'tool_usage_rate': sum(e['metrics']['tool_usage'] for e in evaluations) / len(evaluations),
    'verification_rate': sum(e['metrics']['verified'] for e in evaluations) / len(evaluations),
    'avg_tool_calls': np.mean([e['num_tool_calls'] for e in evaluations])
}

for key, value in avg_metrics.items():
    print(f"  {key:20s}: {value:.3f}")


EVALUATION

1. Площа трикутника...
   Overall: 0.819 | Tools: True | Verified: True

2. Площа трикутника...
   Overall: 0.818 | Tools: True | Verified: True

3. Площа трикутника...
   Overall: 0.803 | Tools: True | Verified: True

4. Об'єм кулі...
   Overall: 0.788 | Tools: True | Verified: True

5. Об'єм кулі...
   Overall: 0.774 | Tools: True | Verified: True

6. Об'єм кулі...
   Overall: 0.786 | Tools: True | Verified: True

SUMMARY
  overall_score       : 0.798
  retrieval_quality   : 0.790
  ukrainian_ratio     : 0.918
  completeness        : 1.000
  correctness         : 0.000
  structure_rate      : 1.000
  citation_rate       : 0.667
  tool_usage_rate     : 1.000
  verification_rate   : 1.000
  avg_tool_calls      : 1.500


## 8. Save Results

In [16]:
results = {
    'experiment': 'rag_with_tools',
    'description': 'RAG + Wolfram Alpha for verified computations',
    'model': 'gemma-2-9b-it',
    'tools': ['wolfram_alpha'],
    'wolfram_mode': 'DEMO' if WOLFRAM_APP_ID == 'DEMO' else 'API',
    'avg_metrics': avg_metrics,
    'responses': [r.to_dict() for r in responses],
    'evaluations': evaluations
}

with open(OUTPUT_DIR / 'results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Results saved to {OUTPUT_DIR}")
print("\n" + "="*80)
print("EXPERIMENT 4 COMPLETE")
print("="*80)
print(f"\nOverall Score: {avg_metrics['overall_score']:.3f}")
print(f"Tool Usage Rate: {avg_metrics['tool_usage_rate']:.3f}")
print(f"Verification Rate: {avg_metrics['verification_rate']:.3f}")
print(f"Avg Tool Calls: {avg_metrics['avg_tool_calls']:.1f}")

Results saved to ../evaluation/experiment_04

EXPERIMENT 4 COMPLETE

Overall Score: 0.798
Tool Usage Rate: 1.000
Verification Rate: 1.000
Avg Tool Calls: 1.5
